##  Sentinel-2 burned area identification

This notebook prepares the data transformation application: The outcome is a data transformation application that takes one input (or a set of inputs organized in an atomic unit) and generates the output.

The application implements:

* Calculation of NDVI in the two scenes (using band 8 and 4), (B8-B4)/(B8+B4)
* Calculation of NDWI, in the two scenes (using band 8 and 11), (B8-B11)/(B8+B11)
* If NDWI i2 - NDWI i1 > 0.18 and If NDVI i2 - NDVI i1 > 0.19 then burned pixels

The outputs generated include:

* COG RGB composite with bands 12, 11, 8A pre
* COG RGB composite with bands 12, 11, 8A post 
* COG scene classification pre  
* COG scene classification post 
* COG 8 bits with bitmask burned/not burned
* Geojson with vectorization of bitmask burned/not burned

In [1]:
service = dict([('title', 'Sentinel-2 burned area identification'),
                ('abstract', 'This is a short description'),
                ('id', 'ewf-satcen-03-03-02')])

### Parameter Definition 

### Runtime parameter definition

**Input reference**


In [2]:
input_references = ['https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20191231T000241_N0213_R030_T56HKG_20191231T015159',
                   'https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20191201T000241_N0213_R030_T56HKG_20191201T020044']

In [3]:
ndvi_threshold = dict([('identifier', 'ndvi_threshold'),
                       ('value', '0.19'),
                       ('title', 'NDVI difference threshold'),
                       ('abstract', 'NDVI difference threshold'),
                       ('maxOccurs', '1')]) 

In [4]:
 ndwi_threshold = dict([('identifier', 'ndwi_threshold'),
                       ('value', '0.18'),
                       ('title', 'NDWI difference threshold'),
                       ('abstract', 'NDWI difference threshold'),
                       ('maxOccurs', '1')])

In [5]:
wkt = dict([('id', 'aoi'),
            ('value', 'POLYGON ((150.4704663611788 -34.65618387777952, 150.4704663611788 -34.95618387777952, 150.1704663611788 -34.95618387777952, 150.1704663611788 -34.65618387777952, 150.4704663611788 -34.65618387777952))'),
            ('title', 'Area of interest'),
            ('abstract', 'Area of interest in WKT or bounding box')])

**Data path**

This path defines where the data is staged-in. 

In [6]:
data_path = '/workspace/data'

### Workflow

#### Import the packages

In [7]:
import os
import sys
import cioppy
import snappy
from snappy import GPF
ciop = cioppy.Cioppy()
from snappy import jpy
from snappy import ProductIO

import gdal
import geopandas as gp
import numpy as np
import datetime
sys.path.append(os.getcwd())
from helpers import *


gdal.UseExceptions() 



In [8]:
products = get_metadata(input_references, data_path)

In [9]:
group_analysis(products)

In [10]:
products

,identifier,self,startdate,enddate,enclosure,orbitDirection,track,orbitNumber,wkt,local_path,ordinal_type
0,S2A_MSIL2A_20191231T000241_N0213_R030_T56HKG_2...,https://catalog.terradue.com/sentinel2/search?...,2019-12-31T00:02:41.0250000Z,2019-12-31T00:02:41.0250000Z,https://store.terradue.com/download/sentinel2/...,DESCENDING,30,23622,"POLYGON((150.907587083732 -35.3131554422379,15...",/workspace/data/S2A_MSIL2A_20191231T000241_N02...,Pst
1,S2A_MSIL2A_20191201T000241_N0213_R030_T56HKG_2...,https://catalog.terradue.com/sentinel2/search?...,2019-12-01T00:02:41.0240000Z,2019-12-01T00:02:41.0240000Z,https://store.terradue.com/download/sentinel2/...,DESCENDING,30,23193,"POLYGON((150.907587083732 -35.3131554422379,15...",/workspace/data/S2A_MSIL2A_20191201T000241_N02...,Pre


In [11]:
req_bands = ['B4','B8','B8A', 'B11','B12', 'quality_scene_classification' ]

In [12]:
#if more than one post or pre products==> use slice assembly ==>use new mosaics as input to the next steps

In [13]:

for index, item in enumerate(['Pst','Pre']):
    if(products[products['ordinal_type'] == item].identifier.count()>1):
        product = mosaic_inputs(products[products['ordinal_type'] == item].reset_index(drop=True))
    else:
        local_pathx=products[products['ordinal_type'] == item].iloc[0]['local_path']
        s2prd = "%s/MTD_MSIL2A.xml" %local_pathx 
        product = snappy.ProductIO.readProduct(s2prd)
    
    output_name = '%s_%s.tif'%(item,product.getName())
    print(output_name)
    product = resample2ref_band(product,'B4')
    product = subset_to_aoi_reduce_bands(product,wkt['value'],req_bands)
    ProductIO.writeProduct(product, 'S2_%s_tmp.tif'%item, 'GeoTIFF-BigTIFF')
    
    snap_rgb(product,['B12','B11','B8A'],'RGB_%s'%output_name)
    snap_mask(product,'MASK_%s'%output_name)

Pst_S2A_MSIL2A_20191231T000241_N0213_R030_T56HKG_20191231T015159.tif
Pre_S2A_MSIL2A_20191201T000241_N0213_R030_T56HKG_20191201T020044.tif


In [14]:
list(product.getBandNames())

['B4', 'B8', 'B8A', 'B11', 'B12', 'quality_scene_classification']

### NDVI & NDWI computation 

In [15]:
ds = gdal.Open('S2_Pre_tmp.tif')
    
pre_b04 = ds.GetRasterBand(1).ReadAsArray()
pre_b08 = ds.GetRasterBand(2).ReadAsArray()
pre_b11 = ds.GetRasterBand(4).ReadAsArray()
pre_scl = ds.GetRasterBand(6).ReadAsArray()

ds = None

#os.remove('pre_event.tif')

In [16]:
ds = gdal.Open('S2_Pst_tmp.tif')
    
post_b04 = ds.GetRasterBand(1).ReadAsArray()
post_b08 = ds.GetRasterBand(2).ReadAsArray()
post_b11 = ds.GetRasterBand(4).ReadAsArray()
post_scl = ds.GetRasterBand(6).ReadAsArray()

width = ds.RasterXSize
height = ds.RasterYSize

input_geotransform = ds.GetGeoTransform()
input_georef = ds.GetProjectionRef()
#print(input_georef)
proj = osr.SpatialReference(wkt=ds.GetProjection())
espg = proj.GetAttrValue('AUTHORITY',1)
print(espg) 
ds = None

#os.remove('post_event.tif')

32756


In [17]:
gain = 10000

 ### NDWI with NIR (8) and SWIR (11)

In [18]:
pre_ndwi2 = (pre_b08 / gain - pre_b11 / gain) / (pre_b08/ gain / gain + pre_b11 / gain)
post_ndwi2 = (post_b08 / gain - post_b11 / gain) / (post_b08/ gain / gain + post_b11 / gain)

pre_b11 = None
post_b11 = None

/opt/anaconda/envs/env_ewf_satcen_03_03_02/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


### NDVI with NIR (8) and Red (4)

In [19]:
pre_ndvi = (pre_b08 / gain - pre_b04 / gain) / (pre_b08 / gain / gain + pre_b04 / gain)
post_ndvi = (post_b08 / gain - post_b04 / gain) / (post_b08 / gain / gain + post_b04 / gain)

/opt/anaconda/envs/env_ewf_satcen_03_03_02/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [20]:
pre_b04 = None
post_b04 = None

pre_b08 = None
post_b08 = None

In [21]:
conditions = (((post_ndwi2 - pre_ndwi2)  > float(ndwi_threshold['value'])) & ((post_ndvi - pre_ndvi) > float(ndvi_threshold['value'])) & (pre_scl == 4) | (post_scl == 4)) 

/opt/anaconda/envs/env_ewf_satcen_03_03_02/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.


In [22]:
burned = np.zeros((height, width), dtype=np.uint8)

In [23]:
burned[conditions] = 1

In [24]:
pre_ndwi2 = None
post_ndwi2 = None

pre_ndvi = None
post_ndvi = None

### Exclude according to scene classifications:

where noData put burned=2 if burn then put burned=1 else burned=0

In [25]:
burned[np.where((pre_scl == 0) | (post_scl == 0) | (pre_scl == 1) | (post_scl == 1) | (pre_scl == 5) | (post_scl == 5) | (pre_scl == 6) | (post_scl == 6) | (pre_scl == 7) | (post_scl == 7) | (pre_scl == 8) | (post_scl == 8) | (pre_scl == 9) | (post_scl == 9))] = 2

Write

In [26]:
output_name = 'S2_BURNED_AREA.tif'

In [27]:
write_tif(burned, output_name, width, height, input_geotransform, input_georef)

In [28]:
change_detection_gp = polygonize('S2_BURNED_AREA.tif', 1,espg )

/opt/anaconda/envs/env_ewf_satcen_03_03_02/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


#### if we replace {'init':'epsg:{}'.format(epsg)} with new recommended 'epsg:{}', the axis order changes

In [29]:
change_detection_gp.head(10)

,geometry,change_detection
0,"POLYGON ((240680.000 6162180.000, 240680.000 6...",1
1,"POLYGON ((241880.000 6162180.000, 241880.000 6...",2
2,"POLYGON ((242360.000 6162180.000, 242360.000 6...",1
3,"POLYGON ((242500.000 6162180.000, 242500.000 6...",2
4,"POLYGON ((242860.000 6162180.000, 242860.000 6...",2
5,"POLYGON ((243320.000 6162180.000, 243320.000 6...",1
6,"POLYGON ((243360.000 6162180.000, 243360.000 6...",1
7,"POLYGON ((246120.000 6162180.000, 246120.000 6...",1
8,"POLYGON ((247560.000 6162180.000, 247560.000 6...",1
9,"POLYGON ((247780.000 6162180.000, 247780.000 6...",1


### MetaData...

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.